<a href="https://colab.research.google.com/github/rposhala/Recommender-System-on-MovieLens-dataset/blob/main/Recommender_System_using_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATASET_LINK='http://files.grouplens.org/datasets/movielens/ml-100k.zip'

In [3]:
!wget -nc http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -n ml-100k.zip

--2020-10-29 14:41:19--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  16.2MB/s    in 0.3s    

2020-10-29 14:41:19 (16.2 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

#Loading MovieLens dataset

Loading u.info -- The number of users, items, and ratings in the u data set.

In [4]:
overall_stats = pd.read_csv('ml-100k/u.info', header=None)
print("Details of users, items and ratings involved in the loaded movielens dataset: ",list(overall_stats[0]))

Details of users, items and ratings involved in the loaded movielens dataset:  ['943 users', '1682 items', '100000 ratings']


Loading u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.

---



              Each user has rated at least 20 movies.  Users and items are
              numbered consecutively from 1.  The data is randomly ordered. This is a tab separated list of 
	         user id | item id | rating | timestamp. 
              The time stamps are unix seconds since 1/1/1970 UTC 

In [5]:
## same item id is same as movie id, item id column is renamed as movie id
column_names1 = ['user id','movie id','rating','timestamp']
dataset = pd.read_csv('ml-100k/u.data', sep='\t',header=None,names=column_names1)
dataset.head() 

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
len(dataset), max(dataset['movie id']),min(dataset['movie id'])

(100000, 1682, 1)

Loading u.item     -- Information about the items (movies); this is a tab separated

              list of
              movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western |
              The last 19 fields are the genres, a 1 indicates the movie
              is of that genre, a 0 indicates it is not; movies can be in
              several genres at once.
              The movie ids are the ones used in the u.data data set.


In [7]:
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
print(column_names2)

['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [8]:
items_dataset = pd.read_csv('ml-100k/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
items_dataset

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
movie_dataset = items_dataset[['movie id','movie title']]
movie_dataset.head()

,movie id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Looking at length of original items_dataset and length of unique combination of rows in items_dataset after removing movie id column

In [10]:
## looking at length of original items_dataset and length of unique combination of rows in items_dataset after removing movie id column
len(items_dataset.groupby(by=column_names2[1:])),len(items_dataset)

(1664, 1682)

We can see there are 18 extra movie id's for already mapped movie title and the same duplicate movie id is assigned to the user in the user-item dataset.

#Merging required datasets

In [11]:
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='movie id')
merged_dataset.head()

,user id,movie id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


A dataset is created from the existing merged dataset by grouping the unique user id and movie title combination and the ratings by a user to the same movie in different instances (timestamps) are averaged and stored in the new dataset.

Example of a multiple rating scenario by an user to a specific movie:

In [12]:
merged_dataset[(merged_dataset['movie title'] == 'Chasing Amy (1997)') & (merged_dataset['user id'] == 894)]

,user id,movie id,rating,timestamp,movie title
4800,894,246,4,882404137,Chasing Amy (1997)
22340,894,268,3,879896041,Chasing Amy (1997)


## Creating a final refined dataset with unique user id, movie name combination and their ratings:


In [13]:
refined_dataset = merged_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})

refined_dataset.head()

,user id,movie title,rating
0,1,101 Dalmatians (1996),2.0
1,1,12 Angry Men (1957),5.0
2,1,"20,000 Leagues Under the Sea (1954)",3.0
3,1,2001: A Space Odyssey (1968),4.0
4,1,"Abyss, The (1989)",3.0


## Creating lists for unique user id's and movie names:

In [14]:
#list of all users
unique_users = refined_dataset['user id'].unique() 
#creating a list of all movie names in it
unique_movies = refined_dataset['movie title'].unique()
len(unique_movies),len(unique_users)

(1664, 943)

## Converting user id, movie name column of refined dataset to respective lists:

In [15]:
users_list = refined_dataset['user id'].tolist()
movie_list = refined_dataset['movie title'].tolist()
len(users_list),len(movie_list)

(99693, 99693)

## Extracting ratings into a list:

In [16]:
ratings_list = refined_dataset['rating'].tolist()
print(ratings_list)
len(ratings_list)

[2.0, 5.0, 3.0, 4.0, 3.0, 3.0, 1.0, 4.0, 4.0, 5.0, 5.0, 1.0, 5.0, 4.0, 5.0, 3.0, 4.0, 2.0, 4.0, 4.0, 1.0, 5.0, 2.0, 3.0, 1.0, 1.0, 1.0, 3.0, 2.0, 3.0, 5.0, 2.0, 4.0, 5.0, 4.0, 5.0, 3.0, 4.0, 5.0, 1.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 4.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 3.0, 5.0, 3.0, 3.0, 4.0, 5.0, 5.0, 1.0, 3.0, 5.0, 5.0, 5.0, 4.0, 2.0, 4.0, 4.0, 3.0, 2.0, 4.0, 5.0, 2.0, 5.0, 4.0, 5.0, 1.0, 3.0, 4.0, 5.0, 1.0, 3.0, 4.0, 4.0, 3.0, 1.0, 3.0, 4.0, 3.0, 1.0, 1.0, 5.0, 4.0, 3.0, 4.0, 3.0, 5.0, 5.0, 1.0, 3.0, 2.0, 4.0, 5.0, 4.0, 3.0, 4.0, 3.0, 5.0, 4.0, 5.0, 3.0, 4.0, 5.0, 5.0, 2.0, 5.0, 2.0, 1.0, 5.0, 5.0, 4.0, 5.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 5.0, 2.0, 2.0, 1.0, 5.0, 3.0, 5.0, 5.0, 1.0, 4.0, 2.0, 4.0, 3.0, 5.0, 3.0, 2.0, 3.0, 3.0, 2.0, 4.0, 5.0, 5.0, 4.0, 3.0, 5.0, 4.0, 5.0, 2.0, 3.0, 4.0, 5.0, 5.0, 5.0, 3.0, 3.0, 1.0, 2.0, 5.0, 2.0, 3.0, 3.0, 5.0, 1.0, 5.0, 4.0, 1.0, 3.0, 3.0, 3.0, 5.0, 4.0, 4.0, 5.0, 5.0, 5.0, 2.0, 5.0, 4.0, 4.0, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 4.0, 5.0, 4.0, 3.0, 5.0, 4.0,

99693

## Creating a dictionary to map movie name to their corresponding index in the unique movie name list

In [17]:
movies_dict = {unique_movies[i] : i for i in range(len(unique_movies))}
print(movies_dict)
print(len(movies_dict))

{'101 Dalmatians (1996)': 0, '12 Angry Men (1957)': 1, '20,000 Leagues Under the Sea (1954)': 2, '2001: A Space Odyssey (1968)': 3, 'Abyss, The (1989)': 4, 'Ace Ventura: Pet Detective (1994)': 5, 'Air Bud (1997)': 6, 'Akira (1988)': 7, 'Aladdin (1992)': 8, 'Alien (1979)': 9, 'Aliens (1986)': 10, 'All Dogs Go to Heaven 2 (1996)': 11, 'Amadeus (1984)': 12, 'Angels and Insects (1995)': 13, "Antonia's Line (1995)": 14, 'Apocalypse Now (1979)': 15, 'Apollo 13 (1995)': 16, 'Aristocats, The (1970)': 17, 'Army of Darkness (1993)': 18, 'Austin Powers: International Man of Mystery (1997)': 19, 'Babe (1995)': 20, 'Back to the Future (1985)': 21, 'Bad Boys (1995)': 22, 'Basic Instinct (1992)': 23, 'Batman & Robin (1997)': 24, 'Batman Forever (1995)': 25, 'Batman Returns (1992)': 26, 'Beavis and Butt-head Do America (1996)': 27, 'Bedknobs and Broomsticks (1971)': 28, 'Belle de jour (1967)': 29, 'Big Night (1996)': 30, 'Billy Madison (1995)': 31, 'Birdcage, The (1996)': 32, 'Blade Runner (1982)': 33

## Creating a Utility matrix with rows as movies, columns as users, to make the refined dataframe compatible for SVD operations

In [18]:
## creating a utility matrix for the available data

## Creating an empty array with (number of rows = number of movies) and (number of columns = number of users) rows as movies, columns as users

utility_matrix = np.asarray([[np.nan for j in range(len(unique_users))] for i in range(len(unique_movies))])
print("Shape of Utility matrix: ",utility_matrix.shape)

for i in range(len(ratings_list)):

  ## ith entry in users list and subtract 1 to get the index, we do the same for movies but we already defined a dictionary to get the index.
  utility_matrix[movies_dict[movie_list[i]]][users_list[i]-1] = ratings_list[i]

utility_matrix

Shape of Utility matrix:  (1664, 943)


array([[ 2., nan, nan, ..., nan, nan, nan],
       [ 5., nan, nan, ..., nan, nan, nan],
       [ 3., nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

## Normalizing the utility matrix across movies column

In [19]:
mask = np.isnan(utility_matrix)
masked_arr = np.ma.masked_array(utility_matrix, mask)
temp_mask = masked_arr.T
rating_means = np.mean(temp_mask, axis=0)

filled_matrix = temp_mask.filled(rating_means)
filled_matrix = filled_matrix.T
filled_matrix = filled_matrix - rating_means.data[:,np.newaxis]

In [21]:
filled_matrix = filled_matrix.T / np.sqrt(len(movies_dict)-1)
filled_matrix

array([[-0.02227217,  0.01608636, -0.01226094, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

Mean values across movies columns are extracted.  
Nan's of utility matrix are imputed with mean values extracted.  
Later the utility matrix has been normalized across movies to get all ratings to a standard/normal scale.

In [22]:
filled_matrix.shape

(943, 1664)

## Computing SVD (Singular Value Decomposition) of Utility matrix

In [23]:
## Computing the SVD of the input matrix

U, S, V = np.linalg.svd(filled_matrix)

## Creating a list of Case insensitive movie names for further use

In [39]:
case_insensitive_movies_list = [i.lower() for i in unique_movies]

## Defining a function to calculate the cosine similarity on the given dataframe and extracting requesting number of closely matched movie indices with the help of numpy einsum which valuates the Einstein summation convention on the operands.

In [28]:
#Function to calculate the cosine similarity (sorting by most similar and returning the top N)
def top_cosine_similarity(data, movie_id, top_n=10):
  index = movie_id 
  movie_row = data[index, :]
  magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
  similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
  sort_indexes = np.argsort(-similarity)
  return sort_indexes[:top_n]

## Defining a function to get similar movies for the given movie name

In [49]:
#k-principal components to represent movies, movie_id to find recommendations, top_n print n results        
def get_similar_movies(movie_name,top_n,k = 50):
  # k = 50
  # movie_id = 1
  # top_n = 10
  
  sliced = V.T[:, :k] # representative data
  movie_id = movies_dict[movie_name]
  indexes = top_cosine_similarity(sliced, movie_id, top_n)
  print(" ")
  print("Top",top_n,"movies which are very much similar to the Movie-",movie_name, "are: ")
  print(" ")
  for i in indexes[1:]:
    print(unique_movies[i])

In SVD decomposition, Where A is a m x n utility matrix, U is a m x r orthogonal left singular matrix, which represents the relationship between users and latent factors, S is a r x r diagonal matrix, which describes the strength of each latent factor and V is a r x n diagonal right singular matrix, which indicates the similarity between items and latent factors. The latent factors here are the characteristics of the items, for example, the genre of the music. The SVD decreases the dimension of the utility matrix A by extracting its latent factors. It maps each user and each item into a r-dimensional latent space. This mapping facilitates a clear representation of relationships between users and items. 

**Dynamically suggesting** movie name from the existing movie corpus we have, based on the user input using try and except architecture.

Defining a function which outputs movie names as suggestion when the user mis spells the movie name. **User might have intended to type any of these movie names.**

In [50]:
# function which takes input and returns suggestions for the user

def get_possible_movies(movie):

    temp = ''
    possible_movies = case_insensitive_movies_list.copy()
    for i in movie :
      out = []
      temp += i
      for j in possible_movies:
        if temp in j:
          out.append(j)
      if len(out) == 0:
          return possible_movies
      out.sort()
      possible_movies = out.copy()

    return possible_movies

This function provides user with **movie name suggestions if movie name is mis-spelled** or **Recommends similar movies to the input movie** if the movie name is valid.

In [46]:
class invalid(Exception):
    pass

def recommender():
    
    try:

      movie_name = input("Enter the Movie name: ")
      movie_name_lower = movie_name.lower()
      if movie_name_lower not in case_insensitive_movies_list :
        raise invalid
      else :
        # movies_list[case_insensitive_country_names.index(movie_name_lower)]
        num_recom = int(input("Enter Number of movie recommendations needed: "))
        get_similar_movies(unique_movies[case_insensitive_movies_list.index(movie_name_lower)],num_recom)

    except invalid:

      possible_movies = get_possible_movies(movie_name_lower)

      if len(possible_movies) == len(unique_movies) :
        print("Movie name entered is does not exist in the list ")
      else :
        indices = [case_insensitive_movies_list.index(i) for i in possible_movies]
        print("Entered Movie name is not matching with any movie from the dataset . Please check the below suggestions :\n",[unique_movies[i] for i in indices])
        print("")
        recommender()


In [47]:
recommender()

Enter the Movie name: dal
Entered Movie name is not matching with any movie from the dataset . Please check the below suggestions :
 ['101 Dalmatians (1996)', 'Mrs. Dalloway (1997)']
Enter the Movie name: 101 Dalmatians (1996)
Enter Number of movie recommendations needed: 10
Top 10 movies which are very much similar to the Movie- 101 Dalmatians (1996) are: 
 
Black Beauty (1994)
Free Willy 2: The Adventure Home (1995)
Evening Star, The (1996)
Robin Hood: Men in Tights (1993)
Cool Runnings (1993)
Turbo: A Power Rangers Movie (1997)
Remains of the Day, The (1993)
City Hall (1996)
Children of the Corn: The Gathering (1996)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


## Conclusion:

The above built recommender system can suggest movie names similar to the given movie name. The same architecture can be built to get similar users for the given user id and then suggesting movies from those user id's (this would be item-item collaborative filtering). This can be done just by extracting latent factors of U matrix (which corresponds to Users) instead V matrix (which corresponds to Movies) as we did. But a better recommender system can be built only if we could change the axis of normalization of utility matrix to userid from movies.

# Rough Work

In [78]:
movies_dict['101 Dalmatians (1996)']

0

In [48]:
len(movie_list)

99693

In [58]:
util_mat = np.asarray([[np.nan for j in range(4)] for i in range(7)])
util_mat[6][3] = 4
util_mat[2][3] = 12
util_mat[6][0] = 1
util_mat[4][1] = 4
util_mat[0][2] = 8
util_mat[5][1] = 2
util_mat[6][3] = 3
util_mat[3][1] = 6
util_mat[5][0] = 10
util_mat[1][3] = 2
util_mat[0][2] = 7
util_mat[1][0] = 13
util_mat

array([[nan, nan,  7., nan],
       [13., nan, nan,  2.],
       [nan, nan, nan, 12.],
       [nan,  6., nan, nan],
       [nan,  4., nan, nan],
       [10.,  2., nan, nan],
       [ 1., nan, nan,  3.]])

In [59]:
mask = np.isnan(util_mat)
masked_arr = np.ma.masked_array(util_mat, mask)
transp = masked_arr.T
item_means = np.mean(transp, axis=0)
# np.mean(utility_matrix,axis=0)
s = transp.filled(item_means)
s = s.T
s

array([[ 7. ,  7. ,  7. ,  7. ],
       [13. ,  7.5,  7.5,  2. ],
       [12. , 12. , 12. , 12. ],
       [ 6. ,  6. ,  6. ,  6. ],
       [ 4. ,  4. ,  4. ,  4. ],
       [10. ,  2. ,  6. ,  6. ],
       [ 1. ,  2. ,  2. ,  3. ]])

In [50]:
e = item_means.data
e

array([ 7. ,  7.5, 12. ,  6. ,  4. ,  6. ,  2. ])

In [56]:
e = e[:,np.newaxis]

In [57]:
s.T - e

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 5.5,  0. ,  0. , -5.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 4. , -4. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  1. ]])

In [36]:
d = np.asarray([[2,3,5],[5,6,8]])
d - [[2],[5]]

array([[0, 1, 3],
       [0, 1, 3]])

In [44]:
f = np.asarray([2,5])
f.T

array([2, 5])

In [37]:
k = np.asarray([[2,3,5],[5,6,8],[6,3,1]])
k

array([[2, 3, 5],
       [5, 6, 8],
       [6, 3, 1]])

In [38]:
np.einsum('ij, ij -> i', k, k)

array([ 38, 125,  46])

In [45]:
np.einsum('ii, ii', k, k)

41

In [46]:
a = [[1,3],[4,6]]
np.einsum('ij, ij -> i', a, a)

array([10, 52])